In [ ]:
import autograd.numpy as np
import autograd
import os
from autograd import grad
from autograd import jacobian
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.linalg import pinv
from numpy import linalg 

from cubic_reg import CubicRegularization


In [ ]:

def f1(z):
    x = z[0]
    y = z[1]
    f = -3*x*x-y*y+4*x*y
    return f

def f2(z):
    x = z[0]
    y = z[1]
    f = 3*x*x+y*y+4*x*y
    return f

def f3(z):
    x = z[0]
    y = z[1]
    f = (0.4*x*x-0.1*(y-3*x+0.05*x*x*x)**2-0.01*y*y*y*y)*np.exp(-0.01*(x*x+y*y))
    return f


def f4(z):
    x = z[0]
    y = z[1]
    f = 2*x*x + 0.5*y*y - 4*x*y + (4/3)*y*y*y - (1/4)*y*y*y*y
    return f
    
## Test it matches with paper
# grad_fn = grad(f4)
# hessian_fn = jacobian(grad_fn)

# z0 = np.array([0.,0.])
# z1 = np.array([1.,1.])
# z2 = np.array([3.,3.])
# print(grad_fn(z0), grad_fn(z1), grad_fn(z2))
# hessian_fn(z0), hessian_fn(z1), hessian_fn(z2)

In [ ]:

# GDA
def gda(z_0, alpha=0.05, num_iter=100, grad_fn=None):
    z = [z_0]
    if not grad_fn:
        grad_fn = grad(target)
    for i in range(num_iter):
        g = grad_fn(z[-1])
        z1 = z[-1] + g*np.array([-1,1])*alpha
        z.append(z1)
    z = np.array(z)
    return z

# Extra Gradient
def eg(z_0, alpha=0.05, num_iter=100, grad_fn=None):
    z = [z_0]
    if not grad_fn:
        grad_fn = grad(target)
    for i in range(num_iter):
        g = grad_fn(z[-1])
        z1 = z[-1] + g*np.array([-1,1])*alpha
        g = grad_fn(z1)
        z2 = z[-1] + g*np.array([-1,1])*alpha
        z.append(z2)
    z = np.array(z)
    return z

# Optimistic Gradient
def ogda(z_0, alpha=0.05, num_iter=100):
    z = [z_0,z_0]
    grads = []
    grad_fn = grad(target)
    for i in range(num_iter):
        g = grad_fn(z[-1])
        gg = grad_fn(z[-2])
        z1 = z[-1] + 2*g*np.array([-1,1])*alpha - gg*np.array([-1,1])*alpha
        z.append(z1)
    z = np.array(z)
    return z

# Consensus Optimization
def co(z_0, alpha=0.01, gamma=0.1, num_iter=100):
    z = [z_0]
    grads = []
    grad_fn = grad(target)
    hessian = jacobian(grad_fn)
    for i in range(num_iter):
        g = grad_fn(z[-1])
        H = hessian(z[-1])
        #print(np.matmul(H,g), z[-1])
        v = g*np.array([1,-1]) + gamma*np.matmul(H,g)
        z1 = z[-1] - alpha*v
        z.append(z1)
    z = np.array(z)
    return z

# Symplectic gradient adjustment
def sga(z_0, alpha=0.05, lamb=0.1, num_iter = 100):
    z = [z_0]
    grad_fn = grad(target)
    hessian = jacobian(grad_fn)
    for i in range(num_iter):
        g = grad_fn(z[-1])
        w = g * np.array([1,-1])
        H = hessian(z[-1])
        HH = np.array([[1, -lamb*H[0,1]],[lamb*H[0,1],1]])
        v = HH @ w
        z1 = z[-1] - alpha*v
        z.append(z1)
    z = np.array(z)
    return z

# Follow the ridge
def follow(z_0, alpha=0.05, num_iter = 100):
    z = [z_0]
    grad_fn = grad(target)
    hessian = jacobian(grad_fn)
    for i in range(num_iter):
        g = grad_fn(z[-1])
        H = hessian(z[-1])
        v = np.array([g[0], -g[1]-H[0,1]*np.squeeze(pinv(H[1:,1:]))*g[0]])
        z1 = z[-1] - alpha*v
        z.append(z1)
    z = np.array(z)
    return z


def cubic(z_0, alpha=0.05, num_iter = 100):
    
    grad_fn = grad(target)
    hessian = jacobian(grad_fn)
    
    cr = CubicRegularization(z_0, target, grad_fn, hessian, maxiter=num_iter)
    _, z, _, _ = cr.cubic_reg()
    z = np.array(z)
    return z




def cga(z_0, alpha=0.01, num_iter = 100, L = 0.01, reg='quad'):
    # L: lipschitz constant for quadratic/cubic regularized objective
    
    z = [z_0]
    grad_fn = grad(target)
    hessian = jacobian(grad_fn)
    
    num_iter_subproblem = 50
    
    for i in range(int(num_iter/num_iter_subproblem)):
        g = grad_fn(z[-1])
        H = hessian(z[-1])

        if reg == 'quad':
            def subproblem_f(z):
                x = z[0]
                y = z[1]
                return x*g[0] + y*g[1] + 0.5*x*H[0,1]*y + 0.5*x*H[1,0]*y + (L/2)*(x*x) - (L/2)*(y*y)
        elif reg == 'cubic':
            def subproblem_f(z):
                x = z[0]
                y = z[1]
                return x*g[0] + y*g[1] + 0.5*x*H[0,1]*y + 0.5*x*H[1,0]*y + (L/6)*(x*x*x) - (L/6)*(y*y*y)
            
        subproblem_gx = grad( subproblem_f)
        subproblem_gy = grad(lambda z: -subproblem_f(z))
        def subproblem_grad(z):
            return np.array([
                subproblem_gx(z)[0], subproblem_gy(z)[1]
            ])

        subproblem_z = eg(np.array([0.,0.]), alpha=alpha, num_iter=50, grad_fn=subproblem_grad)
        z.append(subproblem_z[-1])

    z = np.array(z)
    return z

In [ ]:
function = 4


# Select target function
if function==1:
    target = f1              # (0,0) is local minimax and global minimax
    z_0 = np.array([5., 7.]) # Set initial point
    plot_width = 12          # Set range of the plot
    root_dir = 'results/f1.png'
    f_title = 'divergent'
elif function==2:
    target = f2         # (0,0) is not local minimax and not global minimax
    z_0 = np.array([6., 5.])
    plot_width = 12
    root_dir = 'results/f2.png'
    f_title = 'bad stable point (not local minimax)'
elif function==3:
    target = f3         # (0,0) is local minimax
    z_0 = np.array([7., 5.])
    plot_width = 8
    root_dir = 'results/f3.png'
    f_title = 'limiting cycle'
elif function==4:
    target = f4  
    # (0,0) locally stable not nash 
    # (1,1) locally unstable not local nash
    # (3,3) is only second order nash, locally stable
    z_0 = np.array([3., -1.])
    plot_width = 4
    root_dir = 'results/f4.png'
    f_title = '3 1st order local saddle, 1 true saddle'
    

In [ ]:
# Run all algorithms on target
n_iterations = 1000
zfr=follow(z_0, num_iter = n_iterations, alpha = 0.001)
zgda=gda(z_0, num_iter = n_iterations, alpha = 0.05)
zogda=ogda(z_0, num_iter = n_iterations, alpha = 0.05)
zeg=eg(z_0, num_iter = n_iterations, alpha = 0.05)
zco=co(z_0, num_iter = n_iterations, alpha = 0.01, gamma=0.1)
zsga=sga(z_0, num_iter = n_iterations, alpha = 0.01, lamb=1.0)
zcubic=cubic(z_0, num_iter = n_iterations, alpha = 0.01)
zcgaquad = cga(z_0, alpha=0.001, num_iter = n_iterations, L = 0.01,reg='quad')
zcgacubic = cga(z_0, alpha=0.001, num_iter = n_iterations, L = 0.01,reg='cubic')

In [ ]:
# Plot trajectory with contour
plt.rcParams.update({'font.size': 14})
def_colors=(plt.rcParams['axes.prop_cycle'].by_key()['color'])


#plot_width=12
plt.figure(figsize=(10,10))
axes = plt.gca()
axes.set_xlim([-plot_width,plot_width])
axes.set_ylim([-plot_width,plot_width])

x1 = np.arange(-plot_width,plot_width,0.1)
y1 = np.arange(-plot_width,plot_width,0.1)
X,Y = np.meshgrid(x1,y1)
Z = np.zeros_like(X)
for i in range(len(x1)):
    for j in range(len(y1)):
        Z[j][i] = target(np.array([x1[i] ,y1[j]]))
plt.contourf(X,Y,Z,100)

lw = 2
hw = 0.7
line6,=plt.plot(zfr[:,0],zfr[:,1],'--',color='r',linewidth=lw,zorder=10)
line1,=plt.plot(zgda[:,0],zgda[:,1],'--',linewidth=lw,color=def_colors[9],zorder=2)
line2,=plt.plot(zogda[:,0],zogda[:,1],'--',linewidth=lw,color=def_colors[1])
line3,=plt.plot(zeg[:,0],zeg[:,1],'--',linewidth=lw,color=def_colors[2])
line4,=plt.plot(zsga[:,0],zsga[:,1],'--',color=def_colors[0],linewidth=lw)
line5,=plt.plot(zco[:,0],zco[:,1],'--',color='xkcd:violet',linewidth=lw)
line7,=plt.plot(zcgaquad[:,0],zcgaquad[:,1],'--',color='xkcd:pink',linewidth=lw+1)
line8,=plt.plot(zcgacubic[:,0],zcgacubic[:,1],'--',color='xkcd:purple',linewidth=lw)


init=plt.plot(zfr[0,0],zfr[0,1],'^',zorder=20,ms=12.0,color='r')
plt.legend((line6,line1, line2, line3, line4, line5,line7,line8), ('FR','GDA', 'OGDA', 'EG', 'SGA', 'CO', 'CGD+quad','CGD+cubic'), loc=4)
plt.tight_layout()
plt.axis('off')
plt.title(f_title)

os.makedirs('results/', exist_ok=True)
plt.savefig(root_dir, dpi=300)

In [ ]:
zcga